# Letter recognition (small size)

> Indeed, I once even proposed that the toughest challenge facing AI workers is to answer the question: “What are the letters ‘A’ and ‘I’? - [Douglas R. Hofstadter](https://web.stanford.edu/group/SHR/4-2/text/hofstadter.html) (1995)


## notMNIST


Data source: [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) (you need to download `notMNIST_small.mat` file):

![](http://yaroslavvb.com/upload/notMNIST/nmn.png)

> some publicly available fonts and extracted glyphs from them to make a dataset similar to MNIST. There are 10 classes, with letters A-J taken from different fonts.

> Approaching 0.5% error rate on notMNIST_small would be very impressive. If you run your algorithm on this dataset, please let me know your results.


## So, why not MNIST?

Many introductions to image classification with deep learning start with MNIST, a standard dataset of handwritten digits. This is unfortunate. Not only does it not produce a “Wow!” effect or show where deep learning shines, but it also can be solved with shallow machine learning techniques. In this case, plain k-Nearest Neighbors produces more than 97% accuracy (or even 99.5% with some data preprocessing!). Moreover, MNIST is not a typical image dataset – and mastering it is unlikely to teach you transferable skills that would be useful for other classification problems

> Many good ideas will not work well on MNIST (e.g. batch norm). Inversely many bad ideas may work on MNIST and no[t] transfer to real [computer vision]. - [François Chollet’s tweet](https://twitter.com/fchollet/status/852594987527045120)

In [ ]:
!wget http://yaroslavvb.com/upload/notMNIST/notMNIST_small.mat

In [ ]:
import matplotlib.pyplot as plt
from scipy import io
import numpy as np
from sklearn.model_selection import train_test_split

## Data Loading

In [ ]:
data = io.loadmat("notMNIST_small.mat")

# transform data
X = data['images']
y = data['labels']
resolution = 28
classes = 10

X = np.transpose(X, (2, 0, 1))

y = y.astype('int32')
X = X.astype('float32') / 255.

# shape: (sample, x, y, channel)
X = X.reshape((-1, resolution, resolution, 1))

In [ ]:
# looking at data; some fonts are strange
i = np.random.randint(0, 18724)

plt.imshow(X[i,:,:,0])
plt.title("ABCDEFGHIJ"[y[i]]);

In [ ]:
# random letters
rows = 6
fig, axs = plt.subplots(rows, classes, figsize=(classes, rows))

for letter_id in range(10):
    letters = X[y == letter_id]
    for i in range(rows):
        ax = axs[i, letter_id]
        ax.imshow(letters[np.random.randint(len(letters)),:,:,0],
                  cmap='Greys', interpolation='none')
        ax.axis('off')

In [ ]:
# splitting data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
class_n = len(np.unique(y_train))

In [ ]:
y_train = to_categorical(y_train, class_n)
y_test = to_categorical(y_test, class_n)

In [ ]:
x_train.shape, y_train.shape

# Keras를 이용한 모델링 훈련!

1. Flatten layer 활용
2. activation이 주어진 Dense layer뒤에 BatchNormalization둬볼 것
3. Dropout을 0.2 정도로 활용해볼 것
4. Early stopping도 활용해볼 것

In [ ]:
## Sequential API
# 1번. 세션 클리어 : 기존의 모델 구조가 메모리에 남아있다면 지워줘
keras.backend.clear_session()

# 2. 모델 발판 선언 : 레이어 블록을 차곡차곡 쌓을!
model = keras.models.Sequential()

# 3. 레이어 조립 : .add( )
model.add( keras.layers.Input(shape=(28,28,1)) )
model.add( keras.layers.Flatten() )

model.add( keras.layers.Dense(512, activation='relu') )
model.add( keras.layers.Dense(512, activation='relu') )
model.add( keras.layers.BatchNormalization() )
model.add( keras.layers.Dropout(0.2) )

model.add( keras.layers.Dense(256, activation='relu') )
model.add( keras.layers.Dense(256, activation='relu') )
model.add( keras.layers.BatchNormalization() )
model.add( keras.layers.Dropout(0.2) )

model.add( keras.layers.Dense(10, activation='softmax') )

# 4. 컴파일
model.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

In [ ]:
## Functional API
# 1번. 세션 클리어
keras.backend.clear_session()

# 2번. 레이어 사슬처럼 엮기
il = keras.layers.Input(shape=(28,28,1))
fl = keras.layers.Flatten()(il)

hl = keras.layers.Dense(512, activation='relu')(fl)
hl = keras.layers.Dense(512, activation='relu')(hl)
bl = keras.layers.BatchNormalization()(hl)
dl = keras.layers.Dropout(0.2)(bl)

hl = keras.layers.Dense(256, activation='relu')(dl)
hl = keras.layers.Dense(256, activation='relu')(hl)
bl = keras.layers.BatchNormalization()(hl)
dl = keras.layers.Dropout(0.2)(bl)

ol = keras.layers.Dense(10, activation='softmax')(dl)

# 3번. 모델의 시작/끝 지정
model = keras.models.Model(il, ol)

# 4번. 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

In [ ]:
# Early Stopping
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',         ## 관측 대상 : 얼리스토핑을 무엇을 보고 적용할래
                   min_delta=0,                ## 임계값
                   patience=5,                 ## 관측 대상의 성능이 개선되지 않았을 때, 몇 번 더 지켜볼래
                   verbose=1,
                   restore_best_weights=True)  ## 학습하고 나서 최적의 가중치를 가진 epoch의 가중치를 적용!

In [ ]:
model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=10000,verbose=1,
          callbacks=[es])